# Iterators

## Iterator & Iterable

* An iterator is any value that implements `std::iter::Iterator` trait:

In [2]:
mod explain {
    trait Iterator {
        type Item;
        fn next(&mut self) -> Option<Self::Item>;
        //... many default methods
    }
}

* If there’s a natural way to iterate over some type, that type can implement `std::iter::IntoIterator`, whose `into_iter` method takes a value and returns an iterator over it:

In [ ]:
mod explain {
    trait IntoIterator 
        //where Self::IntoIter::Item == Self::Item    
    {
        type Item;
        type IntoIter: Iterator;
        
        fn into_iter(self) -> Self::IntoIter;
    }
}

* An **iterator** is any type that implements `Iterator`.

* An **iterable** is any type that implements `IntoIterator`: you can get an iterator over it by calling its `into_iter` method. The vector reference `&data` is the iterable in this case.

* An iterator **produces** values.

* The values an iterator produces are **items**.

* The code that receives the items an iterator produces is the **consumer** (the for loop is the consumer).

## for loop

In [18]:
let data = vec!["Fender".to_string(), "Gibson".to_string(), "PRS".to_string(), "Yamaha".to_string()];

for element in data {
    println!("{}", element);
}

Fender
Gibson
PRS
Yamaha


()

* for loop is a shorthand for:

In [19]:
{
    let data = vec!["Fender".to_string(), "Gibson".to_string(), "PRS".to_string(), "Yamaha".to_string()];

    let mut iterator = data.into_iter();
    while let Some(element) = iterator.next() {
        println!("{}", element);
    }
}

Fender
Gibson
PRS
Yamaha


()

## Creating iterators

### iter() and iter_mut()

* Most collection types and array slices like `&[T]` and `&mut [T]` provide `iter()` and `iter_mut()` methods that return the natural iterators over the type, producing a shared or mutable reference to each item

In [12]:
{ 
    let data = vec![1, 1, 2, 3, 5, 8, 13];

    let mut iterator = data.iter();
    assert_eq!(iterator.next(), Some(&1));
    assert_eq!(iterator.next(), Some(&1));
    assert_eq!(iterator.next(), Some(&2));
    assert_eq!(iterator.next(), Some(&3));
    assert_eq!(iterator.next(), Some(&5));
    assert_eq!(iterator.next(), Some(&8));
    assert_eq!(iterator.next(), Some(&13));
    assert_eq!(iterator.next(), None);
}

()

* This iterator’s item type is `&i32`: each call to next produces a reference to the next element, until we reach the end of the vector.

### Iteration over &str

* There is no `iter()` method on the `&str` string slice type. Instead, if `s` is a `&str`, then `s.bytes()` returns an iterator that produces each byte of s, whereas `s.chars()` interprets the contents as UTF-8 and produces each Unicode character.

### IntoIterator implementations

* When a type implements `IntoIterator`, you can call its `into_iter()` method yourself, just as a for loop would:

In [13]:
use std::collections::BTreeSet;

let mut guitars = BTreeSet::new();
guitars.insert("Yamaha".to_string());
guitars.insert("Fender".to_string());
guitars.insert("Gibson".to_string());

let mut iterator = guitars.into_iter();
assert_eq!(iterator.next(), Some("Fender".to_string()));
assert_eq!(iterator.next(), Some("Gibson".to_string()));
assert_eq!(iterator.next(), Some("Yamaha".to_string()));
assert_eq!(iterator.next(), None);

* Most collections provide several implementations of `IntoIterator`, for shared references `(&T)`, mutable references `(&mut T)`, and owned values`(T)`:
  * Given a **shared reference to the collection**, `into_iter()` returns an iterator that produces shared references to its items. For example, in the preceding code, `(&favorites).into_iter()` would return an iterator whose `Item` type is `&String`.
  * Given a **mutable reference to the collection**, `into_iter()` returns an iterator that produces mutable references to the items. For example, if vector is some `Vec<String>`, the call `(&mut vector).into_iter()` returns an iterator whose `Item` type is `&mut String`.
  * When passed the **collection by value**, `into_iter()` returns an iterator that takes ownership of the collection and returns items by value; 
    * The items’ ownership moves from the collection to the consumer, and the original collection is consumed in the process. For example, the call `guitars.into_iter()` in the preceding code returns an iterator that produces each string by value; the consumer receives ownership of each string. When the iterator is dropped, any elements remaining in the `BTreeSet` are dropped too, and the set’s now-empty husk is disposed of.

* Since a for loop applies `IntoIterator::into_iter()` to its operand, these three implementations are what create the following idioms for iterating over shared or mutable references to a collection, or consuming the collection and taking ownership of its elements:

In [14]:
use std::collections::BTreeSet;

let mut guitars = vec!["Yamaha".to_string(), "Fender".to_string(), "Gibson".to_string()];

for guitar in &guitars {
    println!("{}", guitar);
}

for guitar in &mut guitars {
    *guitar += "!";
}

for guitar in guitars {
    println!("{}", guitar);
}



Yamaha
Fender
Gibson
Yamaha!
Fender!
Gibson!


()

* Not every type provides all three implementations. 
  * `HashSet`, `BTreeSet` and `BinaryHeap` don’t implement `IntoIterator` on mutable references, since modifying their elements would probably violate the type’s invariants
  * `HashMap` and `BTreeMap` produce mutable reference to their entries’ values, but only shared references to their keys
  * Slices implement two of the three `IntoIterator` variants; since they don’t own their elements, there is no “by value” case

### IntoIterator in Generics

* You can use a bound like `T: IntoIterator` to restrict the type variable `T` to types that can be iterated over. 
* Or, you can write `T: IntoIterator<Item=U>` to further require the iteration to produce a particular type `U`

In [20]:
fn dump<T, U>(t: T)
    where T: IntoIterator<Item = U>,
          U: std::fmt::Debug 
{
    for u in t {
        println!("{:?}", u);
    }
}

In [21]:
let mut guitars = vec!["Yamaha".to_string(), "Fender".to_string(), "Gibson".to_string()];
dump(guitars);

"Yamaha"
"Fender"
"Gibson"


### from_fn

* `std::iter::from_fn` creates an iterator from a closure that returns an `Option<T>`. Each call to the closure produces the next item, and iteration ends when the closure returns `None`.

In [3]:
:dep rand = "0.7"

In [4]:
use rand::random;
use std::iter::from_fn;

let lengths: Vec<f64> = from_fn(|| Some((random::<f64>() - random::<f64>()).abs()))
    .take(10)
    .collect();

In [5]:
lengths

[0.07036691681664464, 0.03995339003697729, 0.25553148877504195, 0.39065180632620633, 0.23992812567998623, 0.7579897840337734, 0.17874099861702408, 0.06943825768509615, 0.2978452897816499, 0.18026300786749283]

In [7]:
fn fibonacci() -> impl Iterator<Item = usize> {
    let mut state = (0, 1);
    
    from_fn(move || {
        let next = state.0;
        state = (state.1, state.0 + state.1);
        Some(next)
    })
}

In [8]:
fibonacci().take(10).collect::<Vec<_>>()

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

### drain methods

* The `drain()` method removes the specified range from the vector in bulk, returning all removed elements as an iterator. If the iterator is dropped before being fully consumed, it drops the remaining removed elements.

In [11]:
let mut data = vec![1, 2, 3, 4, 5, 6, 7, 8, 9, 10];
let inner = data.drain(3..7).collect::<Vec<i32>>();

println!("{:?}", data);
println!("{:?}", inner);

data.drain(1..);
println!("{:?}", data);

[1, 2, 3, 8, 9, 10]
[4, 5, 6, 7]
[1]


### Other iterator sources

#### `std::ops::Range` - `1..10`

* Endpoints must be an integer type to be iterable

In [12]:
(1..10).collect::<Vec<i32>>()

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [13]:
(1..10).step_by(2).collect::<Vec<i32>>()

[1, 3, 5, 7, 9]

#### `std::ops::RangeFrom` - `1..`

* Unbounded iteration. Start must be an integer. May panic or overflow if the value reaches the limit of the type.

In [14]:
(1..).take(10).collect::<Vec<i32>>()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

#### `Option<T>` - `Some(5).iter()` or `None.iter()`

In [15]:
let opt = Some(5);
for v in opt {
    println!("{}", v);
}

let opt: Option<i32> = None;
for v in opt {
    println!("{}", v);
}

5


()

#### `Result<T, E>`

* `iter()` and `iter_mut()` methods are available for `Result<T, E>`
* Returns an iterator over the possibly contained value. The iterator yields one value if the result is `Result::Ok`, otherwise none.

In [33]:
fn load_data(path: &str) -> Result<i32, &str> {
    if path != "not-found" {
        Ok(42)
    } else {
        Err("File not found")
    }
}

let data = load_data("found");
assert_eq!(data.iter().next(), Some(&42));

let data = load_data("not-found");
assert_eq!(data.iter().next(), None);

#### `Vec[T]`, `&[T]`

* `v.windows(3)` - Returns an iterator over contiguous windows of length 3 within the vector.

In [35]:
let v = vec![1, 2, 3, 4, 5, 6, 7, 8, 9, 10];

for wnd in v.windows(3) {
    println!("{:?}", wnd);
}

[1, 2, 3]
[2, 3, 4]
[3, 4, 5]
[4, 5, 6]
[5, 6, 7]
[6, 7, 8]
[7, 8, 9]
[8, 9, 10]


()

* `v.chunks(3)` & `v.chunks_mut(3)` - Return iterators over contiguous chunks of length 3 within the vector, with `chunks_mut` providing mutable references.

In [36]:
for chunk in v.chunks(3) {
    println!("{:?}", chunk);
}

[1, 2, 3]
[4, 5, 6]
[7, 8, 9]
[10]


()

* `v.split(predicate)` & `v.split_mut(predicate)` - Return iterators over subslices separated by elements that match the predicate, with `split_mut` providing mutable references.

In [37]:
for slice in v.split(|&n| n % 3 == 0) {
    println!("{:?}", slice);
}

[1, 2]
[4, 5]
[7, 8]
[10]


()

In [39]:
for slice in v.rsplit(|&n| n % 3 == 0) {
    println!("{:?}", slice);
}

[10]
[7, 8]
[4, 5]
[1, 2]


()

#### `String`, `&str`

In [41]:
let text = "Nobody expects the Spanish Inquisition!\n Amongst our weaponry are such diverse elements as fear, surprise, ruthless efficiency, an almost fanatical devotion to the Pope, and nice red uniforms.".to_string();

* `text.bytes()` - produces bytes of the UTF-8 form

In [42]:
text.bytes()

Bytes(Copied { it: Iter([78, 111, 98, 111, 100, 121, 32, 101, 120, 112, 101, 99, 116, 115, 32, 116, 104, 101, 32, 83, 112, 97, 110, 105, 115, 104, 32, 73, 110, 113, 117, 105, 115, 105, 116, 105, 111, 110, 33, 10, 32, 65, 109, 111, 110, 103, 115, 116, 32, 111, 117, 114, 32, 119, 101, 97, 112, 111, 110, 114, 121, 32, 97, 114, 101, 32, 115, 117, 99, 104, 32, 100, 105, 118, 101, 114, 115, 101, 32, 101, 108, 101, 109, 101, 110, 116, 115, 32, 97, 115, 32, 102, 101, 97, 114, 44, 32, 115, 117, 114, 112, 114, 105, 115, 101, 44, 32, 114, 117, 116, 104, 108, 101, 115, 115, 32, 101, 102, 102, 105, 99, 105, 101, 110, 99, 121, 44, 32, 97, 110, 32, 97, 108, 109, 111, 115, 116, 32, 102, 97, 110, 97, 116, 105, 99, 97, 108, 32, 100, 101, 118, 111, 116, 105, 111, 110, 32, 116, 111, 32, 116, 104, 101, 32, 80, 111, 112, 101, 44, 32, 97, 110, 100, 32, 110, 105, 99, 101, 32, 114, 101, 100, 32, 117, 110, 105, 102, 111, 114, 109, 115, 46]) })

* `text.chars()` - produces `char`s of UTF-8 represents

In [43]:
text.chars()

Chars(['N', 'o', 'b', 'o', 'd', 'y', ' ', 'e', 'x', 'p', 'e', 'c', 't', 's', ' ', 't', 'h', 'e', ' ', 'S', 'p', 'a', 'n', 'i', 's', 'h', ' ', 'I', 'n', 'q', 'u', 'i', 's', 'i', 't', 'i', 'o', 'n', '!', '\n', ' ', 'A', 'm', 'o', 'n', 'g', 's', 't', ' ', 'o', 'u', 'r', ' ', 'w', 'e', 'a', 'p', 'o', 'n', 'r', 'y', ' ', 'a', 'r', 'e', ' ', 's', 'u', 'c', 'h', ' ', 'd', 'i', 'v', 'e', 'r', 's', 'e', ' ', 'e', 'l', 'e', 'm', 'e', 'n', 't', 's', ' ', 'a', 's', ' ', 'f', 'e', 'a', 'r', ',', ' ', 's', 'u', 'r', 'p', 'r', 'i', 's', 'e', ',', ' ', 'r', 'u', 't', 'h', 'l', 'e', 's', 's', ' ', 'e', 'f', 'f', 'i', 'c', 'i', 'e', 'n', 'c', 'y', ',', ' ', 'a', 'n', ' ', 'a', 'l', 'm', 'o', 's', 't', ' ', 'f', 'a', 'n', 'a', 't', 'i', 'c', 'a', 'l', ' ', 'd', 'e', 'v', 'o', 't', 'i', 'o', 'n', ' ', 't', 'o', ' ', 't', 'h', 'e', ' ', 'P', 'o', 'p', 'e', ',', ' ', 'a', 'n', 'd', ' ', 'n', 'i', 'c', 'e', ' ', 'r', 'e', 'd', ' ', 'u', 'n', 'i', 'f', 'o', 'r', 'm', 's', '.'])

* `text.split_whitespace()` - Returns an iterator over the non-whitespace substrings of the string slice.

In [44]:
for text_chunk in text.split_whitespace() {
    println!("{}", text_chunk);
}

Nobody
expects
the
Spanish
Inquisition!
Amongst
our
weaponry
are
such
diverse
elements
as
fear,
surprise,
ruthless
efficiency,
an
almost
fanatical
devotion
to
the
Pope,
and
nice
red
uniforms.


()

* `text.lines()` - Returns an iterator over the lines of the string slice.

In [54]:
for line in text.lines() {
    println!("{}", line.trim());
}

Nobody expects the Spanish Inquisition!
Amongst our weaponry are such diverse elements as fear, surprise, ruthless efficiency, an almost fanatical devotion to the Pope, and nice red uniforms.


()

* `text.matches` - Returns an iterator over all non-overlapping matches of a pattern in the string slice.

In [56]:
let text = "A1B2C3D4".to_string();

for numeric_chunks in text.matches(char::is_numeric) {
    println!("{}", numeric_chunks);
}

1
2
3
4


()

#### `HashMap`, `BTreeMap`

* `HashMap` - A hash table-based implementation of a map, providing fast lookups and insertions on average.
* `BTreeMap` - A tree-based implementation of a map, providing ordered key-value pairs and efficient range queries.

In [45]:
let mut dictionary = std::collections::HashMap::new();

dictionary.insert("apple", 3);
dictionary.insert("banana", 2);
dictionary.insert("cherry", 5);

In [46]:
dictionary.keys()

["apple", "banana", "cherry"]

In [47]:
dictionary.values()

[3, 2, 5]

#### `HashSet`, `BTreeSet`

* `HashSet` - A hash table-based implementation of a set, providing fast lookups and insertions on average.
* `BTreeSet` - A tree-based implementation of a set, providing ordered elements and efficient range queries.

In [48]:
use std::collections::BTreeSet;

let set_a = (1..=10).collect::<BTreeSet<i32>>();
let set_b = (5..=15).collect::<BTreeSet<i32>>();

println!("union: {:?}", set_a.union(&set_b).collect::<Vec<_>>());

println!("intersection: {:?}", set_a.intersection(&set_b).collect::<Vec<_>>());

println!("difference: {:?}", set_a.difference(&set_b).collect::<Vec<_>>());

println!("symmetric_difference: {:?}", set_a.symmetric_difference(&set_b).collect::<Vec<_>>());

union: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
intersection: [5, 6, 7, 8, 9, 10]
difference: [1, 2, 3, 4]
symmetric_difference: [1, 2, 3, 4, 11, 12, 13, 14, 15]


#### Free functions

In [49]:
let empty_iterable = std::iter::empty::<i32>();

let mut iter = empty_iterable.into_iter();

assert_eq!(iter.next(), None);

In [50]:
let one_element_iterable = std::iter::once(42);

let mut iter = one_element_iterable.into_iter();

assert_eq!(iter.next(), Some(42));
assert_eq!(iter.next(), None);

In [51]:
let pattern = std::iter::repeat("*-@").take(9).collect::<String>();
pattern

"*-@*-@*-@*-@*-@*-@*-@*-@*-@"

## Iterator Adaptors

* the `Iterator` trait provides a broad selection of **adapter methods**, or simply adapters, that consume one iterator and build a new one with useful behaviors

### map & filter

* `map` - Transforms each element of an iterator using a closure.
* `filter` - Retains only the elements of an iterator that satisfy a predicate.

In [17]:
let text = "Nobody expects the Spanish Inquisition!\n     Amongst our weaponry are such diverse elements as \n   fear, \n   surprise, \n   ruthless efficiency, \n   an almost fanatical devotion to the Pope, and nice red uniforms.".to_string();

let text_lines = text.lines()
    .map(|s| s.trim().to_owned())
    .collect::<Vec<String>>();

text_lines

["Nobody expects the Spanish Inquisition!", "Amongst our weaponry are such diverse elements as", "fear,", "surprise,", "ruthless efficiency,", "an almost fanatical devotion to the Pope, and nice red uniforms."]

In [18]:
let short_lines = text.lines()
    .map(|s| s.trim().to_owned())
    .filter(|s| s.len() < 20)
    .collect::<Vec<String>>();

short_lines

["fear,", "surprise,"]

* Adapters are lazy

In [21]:
["one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten"]
    .iter()
    .map(|s| println!("{}", s));

### filter_map

* `filter_map` - Combines the functionality of `filter` and `map`, allowing you to both filter and transform elements of an iterator in a single pass.
* Closure passed to `filter_map` takes an element and returns an `Option` type, which determines if the element should be included in the output and how it should be transformed.

In [22]:
let input_data = "1\n2\n3\n4\n5\nabc\n6\n7\ndefn8\n9\n10";

let numbers = input_data.lines()
    .filter_map(|line| line.parse::<i32>().ok())
    .collect::<Vec<i32>>();

numbers

[1, 2, 3, 4, 5, 6, 7, 9, 10]

### flat_map

* `flat_map` - Transforms each element of an iterator and flattens the result into a single iterator.

In [24]:
use std::collections::HashMap;

fn create_major_cities_map() -> std::collections::HashMap<&'static str, Vec<&'static str>> {
    let mut major_cities = std::collections::HashMap::new();
    major_cities.insert("Japan", vec!["Tokyo", "Kyoto"]);
    major_cities.insert("The United States", vec!["Portland", "Nashville"]);
    major_cities.insert("Brazil", vec!["São Paulo", "Brasília"]);
    major_cities.insert("Kenya", vec!["Nairobi", "Mombasa"]);
    major_cities.insert("The Netherlands", vec!["Amsterdam", "Utrecht"]);
    major_cities.insert("Poland", vec!["Warsaw", "Kraków"]);

    major_cities
}

{
    let mut major_cities = create_major_cities_map();
    
    let countries = ["Japan", "Brazil", "Kenya"];
    
    let selected_cities = countries.iter()
        .flat_map(|country| &major_cities[country])
        .collect::<Vec<&&str>>();

    println!("{:?}", selected_cities);
}

["Tokyo", "Kyoto", "São Paulo", "Brasília", "Nairobi", "Mombasa"]


()

### flatten

* `flatten` - Flattens a nested iterator into a single iterator by concatenating all the inner iterators.

In [26]:
{
    let mut major_cities = create_major_cities_map();

    let all_cities: Vec<&&str> = major_cities
    .values()
    .flatten()
    .collect();

    println!("{:?}", all_cities);
}

["Portland", "Nashville", "Amsterdam", "Utrecht", "Warsaw", "Kraków", "Tokyo", "Kyoto", "Nairobi", "Mombasa", "São Paulo", "Brasília"]


()

### take & take_while

* `take(n)` - Returns an iterator that yields the first `n` elements of the original iterator.
* `take_while(predicate)` - Returns an iterator that yields elements from the original iterator as long as the predicate is satisfied.

In [61]:
let message = "To: jimb\r\n\
               From: superego <editor@oreilly.com>\r\n\
               \r\n\
               Did you get any writing done today?\r\n\
               When will you stop wasting time plotting fractals?\r\n";

message.lines().take_while(|l| !l.is_empty()).collect::<Vec<&str>>()

["To: jimb", "From: superego <editor@oreilly.com>"]

### skip & skip_while

* `skip(n)` - Returns an iterator that skips the first `n` elements of the original iterator.
* `skip_while(predicate)` - Returns an iterator that skips elements from the original iterator as long as the predicate is satisfied.

In [62]:
let message = "To: jimb\r\n\
               From: superego <editor@oreilly.com>\r\n\
               \r\n\
               Did you get any writing done today?\r\n\
               When will you stop wasting time plotting fractals?\r\n";

message
    .lines()
    .skip_while(|l| !l.is_empty())
    .skip(1)
    .collect::<Vec<&str>>()

["Did you get any writing done today?", "When will you stop wasting time plotting fractals?"]

### peekable

* A peekable iterator lets you peek at the next item that will be produced without actually consuming it.

In [27]:
use std::iter::Peekable;

fn parse_number<I>(tokens: &mut Peekable<I>) -> i32
    where I: Iterator<Item = char> 
{    
    let mut number = 0;

    loop {
        match tokens.peek() {
            Some(c) if c.is_digit(10) => {
                number = number * 10 + c.to_digit(10).unwrap() as i32;                
            }
            _ => return number
        }
        tokens.next();
    }
}

let mut chars = "423143,353454".chars().peekable();
assert_eq!(parse_number(&mut chars), 423143);
assert_eq!(chars.next(), Some(','));
assert_eq!(parse_number(&mut chars), 353454);
assert_eq!(chars.next(), None);

### fuse

* Once an `Iterator` has returned `None`, the trait doesn’t specify how it ought to behave if you call its next method again. Most iterators just return `None` again, but not all. If your code counts on that behavior, you may be in for a surprise.
* The fuse adapter takes any iterator and produces one that will definitely continue to return `None` once it has done so the first time:

In [28]:
struct Flaky(bool);

impl Iterator for Flaky {
    type Item = &'static str;
    fn next(&mut self) -> Option<Self::Item> {
        if self.0 {
            self.0 = false;
            Some("totally the last item")
        } else {
            self.0 = true; // D'oh!
            None
        }
    }
}

let mut flaky = Flaky(true);
assert_eq!(flaky.next(), Some("totally the last item"));
assert_eq!(flaky.next(), None);
assert_eq!(flaky.next(), Some("totally the last item"));

let mut not_flaky = Flaky(true).fuse();
assert_eq!(not_flaky.next(), Some("totally the last item"));
assert_eq!(not_flaky.next(), None);
assert_eq!(not_flaky.next(), None);

### Reversible Iterators & rev

* The `rev` adapter takes any iterator and produces one that produces the same items in reverse order
* The `DoubleEndedIterator` trait extends `Iterator` with a `next_back` method that returns the last item of the iterator, or `None` if the iterator is empty.

In [66]:
mod explain {
    trait DoubleEndedIterator: Iterator {
        fn next_back(&mut self) -> Option<Self::Item>;
        //... many default methods
    }
}

In [18]:
fn main() {
    let mut data = vec![1, 2, 3, 4];
    
    let mut iterator = data.iter().rev();
    assert_eq!(iterator.next(), Some(&4));
    assert_eq!(iterator.next(), Some(&3));
    assert_eq!(iterator.next(), Some(&2));
    assert_eq!(iterator.next(), Some(&1));
    assert_eq!(iterator.next(), None);
}

main();

### chain

The chain adapter appends one iterator to another. More precisely, `i1.chain(i2)` returns an iterator that draws items from `i1` until it’s exhausted, and then draws items from `i2`.|

In [67]:
let v: Vec<i32> = (1..4).chain(vec![4, 5, 6]).collect();
v

[1, 2, 3, 4, 5, 6]

A `chain` iterator is reversible, if both of its underlying iterators are:

In [68]:
let v_reversed: Vec<i32> = (1..4).chain(vec![4, 5, 6]).rev().collect();
v_reversed

[6, 5, 4, 3, 2, 1]

### enumerate

* `enumerate` - Adds a counter to an iterator and returns it as an iterator of tuples, where each tuple contains the index and the element.

In [69]:
let words = vec!["zero", "one", "two", "three", "four"];

for (index, item) in words.into_iter().enumerate() {
    println!("{}: {}", index, item);
}

0: zero
1: one
2: two
3: three
4: four


()

In [14]:
let v: Vec<(i32, char)> = (0..5).zip("ABCDE".chars()).collect();
v

[(0, 'A'), (1, 'B'), (2, 'C'), (3, 'D'), (4, 'E')]

### by_ref

An iterator’s by_ref method borrows a mutable reference to the iterator, so that you can apply adaptors to the reference. When you’re done consuming items from these adaptors, you drop them, the borrow ends, and you regain access to your original iterator.

In [70]:
let message = "To: SuperUser\r\nFrom: HelpDesk\r\n\r\nHello!\r\n";

let mut lines = message.lines();

for headers in lines.by_ref().take_while(|l| !l.is_empty()) {
    println!("{}", headers);
}

for body in lines {
    println!("{}", body);
}

To: SuperUser
From: HelpDesk
Hello!


()

### cloned

The cloned adapter takes an iterator that produces references, and returns an iterator that produces values cloned from those references. Naturally, the referent type must implement `Clone`

In [71]:
let words = vec!["zero".to_string(), "one".to_string(), "two".to_string(), "three".to_string(), "four".to_string()];

assert_eq!(words.iter().next(), Some(&"zero".to_string()));
assert_eq!(words.iter().cloned().next(), Some("zero".to_string()));

### cycle

The cycle adapter returns an iterator that endlessly repeats the sequence produced by the underlying iterator. The underlying iterator must implement `std::clone::Clone`, so that cycle can save its initial state and reuse it each time the cycle starts again

In [29]:
let directions = ["North", "East", "South", "West"];

for direction in directions.iter().cycle().take(12) {
    println!("{}", direction);
}

North
East
South
West
North
East
South
West
North
East
South
West


()

In [73]:
use std::iter::{once, repeat};

fn main() {
let fizzes = repeat("").take(2).chain(once("fizz")).cycle();
let buzzes = repeat("").take(4).chain(once("buzz")).cycle();
let fizzes_buzzes = fizzes.zip(buzzes);

let fizz_buzz = (1..20).zip(fizzes_buzzes)
    .map(|tuple|
         match tuple {
             (i, ("", "")) => i.to_string(),
             (_, (fizz, buzz)) => format!("{}{}", fizz, buzz)
         });

for line in fizz_buzz {
    println!("{}", line);
}
}

main();

1
2
fizz
4
buzz
fizz
7
8
fizz
buzz
11
fizz
13
14
fizzbuzz
16
17
fizz
19


### count, sum, product

The count method draws items from an iterator until it returns `None`, and tells you how many it got.

In [63]:
let evens_count = (1..100).filter(|&x| x % 2 == 0).count();
evens_count

49

In [42]:
fn triangle(n: u64) -> u64 {
    (1..n+1).sum()
}
assert_eq!(triangle(20), 210);

fn factorial(n: u64) -> u64 {
    (1..n+1).product()
}
assert_eq!(factorial(20), 2432902008176640000);

### max, min

* `max` - Returns the maximum element of an iterator, if it exists.
* `min` - Returns the minimum element of an iterator, if it exists.

In [43]:
assert_eq!([-2, 0, 1, 0, -2, -5].iter().max(), Some(&1));
assert_eq!([-2, 0, 1, 0, -2, -5].iter().min(), Some(&-5));

Rust’s floating-point types `f32` and `f64` implement only `std::cmp::PartialOrd`, `not std::cmp::Ord`, so you can’t use the `min` and `max` methods to compute the least or greatest of a sequence of floating-point numbers. This is not a popular aspect of Rust’s design, but it is deliberate: it’s not clear what such functions should do with IEEE NaN values. Simply ignoring them would risk masking more serious problems in the code.

In [64]:
let real_numbers = vec![1.3, 2.4, 3.5, 4.6, 5.7];
let min_number =real_numbers.iter().min();

Error: the trait bound `{float}: Ord` is not satisfied

### min_by, max_by

The `max_by` and `min_by` methods return the maximum or minimum item the iterator produces, as determined by a comparison function you provide:

In [65]:
use std::cmp::{PartialOrd, Ordering};

// Compare two f64 values. Panic if given a NaN.
fn cmp(lhs: &&f64, rhs: &&f64) -> Ordering {
    lhs.partial_cmp(rhs).unwrap()
}

let numbers = [1.0, 4.0, 2.0];
assert_eq!(numbers.iter().max_by(cmp), Some(&4.0));
assert_eq!(numbers.iter().min_by(cmp), Some(&1.0));

let numbers = [1.0, 4.0, std::f64::NAN, 2.0];
assert_eq!(numbers.iter().max_by(cmp), Some(&4.0)); // panics


thread '<unnamed>' panicked at src\lib.rs:26:26:
called `Option::unwrap()` on a `None` value
stack backtrace:
   0: std::panicking::begin_panic_handler
             at /rustc/1159e78c4747b02ef996e55082b704c09b970588/library\std\src\panicking.rs:697
   1: core::panicking::panic_fmt
             at /rustc/1159e78c4747b02ef996e55082b704c09b970588/library\core\src\panicking.rs:75
   2: core::panicking::panic
             at /rustc/1159e78c4747b02ef996e55082b704c09b970588/library\core\src\panicking.rs:145
   3: core::option::unwrap_failed
             at /rustc/1159e78c4747b02ef996e55082b704c09b970588/library\core\src\option.rs:2130
   4: std::panic::catch_unwind
   5: run_user_code_61
   6: <unknown>
   7: <unknown>
   8: <unknown>
   9: <unknown>
  10: <unknown>
  11: <unknown>
  12: <unknown>
  13: <unknown>
  14: BaseThreadInitThunk
  15: RtlUserThreadStart
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


### min_by_key, max_by_key

The `max_by_key` and `min_by_key` methods are similar to `max_by` and `min_by`, but they take a key extraction function that computes the value to compare:

In [66]:
use std::collections::HashMap;

let mut populations = HashMap::new();
populations.insert("Portland",  583_776);
populations.insert("Fossil",        449);
populations.insert("Greenhorn",       2);
populations.insert("Boring",      7_762);
populations.insert("The Dalles", 15_340);

assert_eq!(populations.iter().max_by_key(|&(_name, pop)| pop),
           Some((&"Portland", &583_776)));
assert_eq!(populations.iter().min_by_key(|&(_name, pop)| pop),
           Some((&"Greenhorn", &2)));

### Comparing Item Sequences

You can use `<` and `==` operators to compare strings, vectors and slices. However, you can’t compare iterators directly.
Iterators provide functions `eq` and `lt` that do comparisons.

In [67]:
let packed = "John Doe,123 Main St.,Hometown,CA,12345";
let spaced = "John Doe,    123 Main St.,   Hometown, CA,    12345";

assert!(packed.split(',').eq(spaced.split(',').map(str::trim)));

### any, all

In [68]:
let sentence = "The quick brown fox jumps over the lazy dog";

assert!(sentence.chars().any(char::is_uppercase));
assert!(sentence.chars().all(|c| c.is_ascii_alphabetic() || c.is_whitespace()));

### position, rposition

The position method applies a closure to each item from the iterator and returns the index of the first item for which the closure returns `true`. More precisely, it returns an `Option` of the index: if the closure returns `true` for no item, position returns `None`. It stops drawing items as soon as the closure returns `true`.

In [70]:
let text = "The quick brown fox jumps over the lazy dog";
assert_eq!(text.chars().position(|c| c.is_whitespace()), Some(3));

let numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 11];
assert_eq!(numbers.iter().rposition(|&n| n % 2 == 0), Some(7));

### fold

The fold method is a very general tool for accumulating some sort of result over the entire sequence of items an iterator produces. Given an initial value, which we’ll call the accumulator, and a closure, fold repeatedly applies the closure to the current accumulator and the next item from the iterator. The value the closure returns is taken as the new accumulator, to be passed to the closure with the next item. The final accumulator value is what fold itself returns. If the sequence is empty, fold simply returns the initial accumulator.

In [72]:
let v = vec![1, 2, 3, 4, 5, 6, 7, 8, 9, 10];

assert_eq!(v.iter().fold(0, |acc, _| acc + 1), 10); // count
assert_eq!(v.iter().fold(0, |acc, n| acc + n), 55); // sum
assert_eq!(v.iter().fold(1, |acc, n| acc * n), 3_628_800); // product

In [73]:
let words = ["And", "now", "for", "something", "completely", "different"];

let sentence = words.iter().fold(String::new(), |mut acc, &word| {
    acc.push_str(word);
    acc.push(' ');
    acc
});

sentence.trim()

"And now for something completely different"

### nth

The `nth` method takes an index `n`, skips that many items from the iterator, and returns the next item, or `None` if the sequence ends before that point. Calling `.nth(0)` is equivalent to `.next()`.

It doesn’t take ownership of the iterator the way an adapter would, so you can call it many times.

In [75]:
fn main() {
    let mut squares = (0..10).map(|n| n * n);

    assert_eq!(squares.nth(4), Some(16));
    assert_eq!(squares.nth(0), Some(25));
    assert_eq!(squares.nth(3), Some(81));
    assert_eq!(squares.nth(2), None);
}

main();

### last

The last method consumes items until the iterator returns `None`, and then returns the last item. If the iterator produces no items, then last returns `None`

In [76]:
fn main() {
    let squares = (0..10).map(|n| n * n);
    assert_eq!(squares.last(), Some(81));
}

This consumes all the iterator’s items starting from the front, even if the iterator is reversible. If you have a reversible iterator and don’t need to consume all its items, you should instead just write `iter.rev().next()`.

### find

The find method draws items from an iterator, returning the first item for which the given closure returns true, or None if the sequence ends before a suitable item is found

In [77]:
fn main() {
    let mut squares = (0..100).map(|n| n * n);

    let first_gt_500 = squares.find(|&n| n > 500);
    println!("{:?}", first_gt_500);
}

main();

Some(529)


### partition

The partition method divides an iterator’s items among two collections, using a closure to decide where each item belongs:

In [39]:
fn main() {
    let numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10];

    let (evens, odds): (Vec<i32>, Vec<i32>) = numbers.iter().partition(|&n| n % 2 == 0);

    println!("even numbers: {:?}", evens);
    println!("odd numbers: {:?}", odds);
}

main();

even numbers: [2, 4, 6, 8, 10]
odd numbers: [1, 3, 5, 7, 9]


## Implementing an Iterator

In [74]:
struct I32Range {
    start: i32,
    end: i32,
}

impl Iterator for I32Range {
    type Item = i32;

    fn next(&mut self) -> Option<Self::Item> {
        if self.start < self.end {
            let result = Some(self.start);
            self.start += 1;
            result
        } else {
            None
        }
    }
}

In [75]:
let mut pi = 0.0;
let mut numerator = 1.0;

for k in (I32Range { start: 0, end: 14 }) {
    pi += numerator / (2*k + 1) as f64;
    numerator /= -3.0;
}
pi *= f64::sqrt(12.0);

// IEEE 754 specifies this result exactly.
assert_eq!(pi as f32, std::f32::consts::PI);

### Case Study - BinaryTree

In [77]:
enum BinaryTree<T> {
    Empty,
    NonEmpty(Box<TreeNode<T>>),
}

struct TreeNode<T> {
    element: T,
    left: BinaryTree<T>,
    right: BinaryTree<T>,
}

struct TreeIter<'a, T> {
    unvisited: Vec<&'a TreeNode<T>>,
}

impl<'a, T> TreeIter<'a, T> {
    fn push_left_edge(&mut self, mut tree: &'a BinaryTree<T>) {
        while let BinaryTree::NonEmpty(ref node) = *tree {
            self.unvisited.push(node);
            tree = &node.left;
        }
    }
}

impl<T> BinaryTree<T> {
    fn iter(&self) -> TreeIter<T> {
        let mut iter = TreeIter { unvisited: Vec::new() };
        iter.push_left_edge(self);
        iter
    }
}

impl<'a, T: 'a> IntoIterator for &'a BinaryTree<T> {
    type Item = &'a T;
    type IntoIter = TreeIter<'a, T>;

    fn into_iter(self) -> Self::IntoIter {
        self.iter()
    }
}

impl<'a, T> Iterator for TreeIter<'a, T> {
    type Item = &'a T;

    fn next(&mut self) -> Option<Self::Item> {
        let node = match self.unvisited.pop() {
            None => return None,
            Some(n) => n,
        };

        self.push_left_edge(&node.right);
        Some(&node.element)
    }
}

In [78]:
use self::BinaryTree::*;

fn make_node<T>(left: BinaryTree<T>, element: T, right: BinaryTree<T>) -> BinaryTree<T> {
    NonEmpty(Box::new(TreeNode { left, element, right }))
}

fn main() {
    // Build a small tree.
    let subtree_l = make_node(Empty, "mecha", Empty);
    let subtree_rl = make_node(Empty, "droid", Empty);
    let subtree_r = make_node(subtree_rl, "robot", Empty);
    let tree = make_node(subtree_l, "Jaeger", subtree_r);

    // Iterate over it.
    let mut v = Vec::new();
    for kind in &tree {
        v.push(*kind);
    }
    assert_eq!(v, ["mecha", "Jaeger", "droid", "robot"]);
}

main();